In [18]:
from dotenv import load_dotenv
from flask import Flask
from flask import render_template
from flask import request
from flask import url_for
import json
import os
import pandas as pd
import pinecone
import re
import requests
from sentence_transformers import SentenceTransformer
from statistics import mean
import swifter

app = Flask(__name__)

PINECONE_INDEX_NAME = "plagiarism-checker"
DATA_FILE = "articles.csv"
NROWS = 20000

def initialize_pinecone():
    load_dotenv()
    os.environ["PINECONE_API_KEY"] = "16210325-84fb-490b-bb1b-8e4be075eb38"
    pinecone.init(api_key="16210325-84fb-490b-bb1b-8e4be075eb38", environment="us-east1-gcp")

def delete_existing_pinecone_index():
    if PINECONE_INDEX_NAME in pinecone.list_indexes():
        pinecone.delete_index(PINECONE_INDEX_NAME)

def create_pinecone_index():
    pinecone.create_index(PINECONE_INDEX_NAME, dimension=8, metric="cosine", shards=1)
    pinecone_index = pinecone.Index(PINECONE_INDEX_NAME)

    return pinecone_index

def create_model():
    model = SentenceTransformer('average_word_embeddings_komninos')

    return model

def prepare_data(data):
    # rename id column and remove unnecessary columns
    data.rename(columns={"Unnamed: 0": "article_id"}, inplace = True)
    data.drop(columns=['date'], inplace = True)

    # combine the article title and content into a single field
    data['content'] = data['content'].fillna('')
    data['content'] = data.content.swifter.apply(lambda x: ' '.join(re.split(r'(?<=[.:;])\s', x)))
    data['title_and_content'] = data['title'] + ' ' + data['content']

    # create a vector embedding based on title and article content
    encoded_articles = model.encode(data['title_and_content'], show_progress_bar=True)
    data['article_vector'] = pd.Series(encoded_articles.tolist())

    return data


#def upload_items(data):
    #items_to_upload = [(row.id, row.article_vector) for i, row in data.iterrows()]
    #vectors = [Vector(id=_id, values=vector) for _id, vector in items_to_upload]
    #vectors = [vector for _id, vector in items_to_upload]
    #pinecone_index.upsert(items=items_to_upload,  vectors=vectors)
    
#def upload_items(data):
    #items_to_upload = list(zip(data.index, data.vector))
    #vectors = []
    #for _id, vector in items_to_upload:
      #  vectors.append(Vector(id=_id, values=vector))
   # pinecone_index.upsert(items=items_to_upload, vectors=vectors)

def upload_items(data):
    if "vector" not in data.columns:
        raise ValueError("DataFrame is missing the 'vector' column.")
    items_to_upload = list(zip(data.index, data.vector))
    vectors = []
    for _id, vector in items_to_upload:
        vectors.append(vector)
    pinecone_index.add(vectors)

#def process_file(filename):
    #data = pd.read_csv(filename, nrows=NROWS)
    #data = prepare_data(data)
    #upload_items(data)
    #pinecone_index.info()

    #return data
    
def process_file(filename):
    if not column_exists(filename, "vector"):
        raise ValueError("The input file does not contain the required 'vector' column.")
    data = pd.read_csv(filename, nrows=NROWS, encoding='UTF-16')
    data = prepare_data(data)
    upload_items(data)
    
    return data

def column_exists(filename, column_name):
    with open(filename, "r") as file:
        header = file.readline().strip().split(",")
        return column_name in header
    
def map_titles(data):
    return dict(zip(uploaded_data.id, uploaded_data.title))

def map_publications(data):
    return dict(zip(uploaded_data.id, uploaded_data.publication))

def query_pinecone(originalContent):
    query_content = str(originalContent)
    query_vectors = [model.encode(query_content)]

    query_results = pinecone_index.query(queries=query_vectors, top_k=10)
    res = query_results[0]

    results_list = []

    for idx, _id in enumerate(res.ids):
        results_list.append({
            "id": _id,
            "title": titles_mapped[int(_id)],
            "publication": publications_mapped[int(_id)],
            "score": res.scores[idx],
        })

    return json.dumps(results_list)

initialize_pinecone()
delete_existing_pinecone_index()
pinecone_index = create_pinecone_index()
model = create_model()
uploaded_data = process_file(filename=DATA_FILE)
titles_mapped = map_titles(uploaded_data)
publications_mapped = map_publications(uploaded_data)

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/api/search", methods=["POST", "GET"])
def search():
    if request.method == "POST":
        return query_pinecone(request.form.get("originalContent", ""))
    if request.method == "GET":
        return query_pinecone(request.args.get("originalContent", ""))
    return "Only GET and POST methods are allowed for this endpoint"

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 1898: character maps to <undefined>

In [10]:
!pip install pandas

In [3]:
!pip install flask

In [4]:
!pip install python-dotenv

  Using cached python_dotenv-0.21.1-py3-none-any.whl (19 kB)


In [6]:
!pip install pinecone-client

  Using cached pinecone_client-2.1.0-py3-none-any.whl (170 kB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached dnspython-2.3.0-py3-none-any.whl (283 kB)
  Using cached loguru-0.6.0-py3-none-any.whl (58 kB)
  Using cached win32_setctime-1.1.0-py3-none-any.whl (3.6 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.0
    Uninstalling tqdm-4.64.0:
      Successfully uninstalled tqdm-4.64.0


In [8]:
!pip install sentence_transformers

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Using cached transformers-4.26.0-py3-none-any.whl (6.3 MB)
  Using cached huggingface_hub-0.12.0-py3-none-any.whl (190 kB)
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125940 sha256=10593f1c238471addc930caf39b3a3b9111c547d663aa95992f1d72f88d7ab92
  Stored in directory: c:\users\elitebook\appdata\local\pip\cache\wheels\71\67\06\162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [10]:
!pip install 

  Using cached swifter-1.3.4.tar.gz (830 kB)
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16321 sha256=d13943d86d0484166264c736bdc1a9ba639193b502d3b98cc158928a85b03371
  Stored in directory: c:\users\elitebook\appdata\local\pip\cache\wheels\2b\5e\f2\3931524f702ffd03309e96d35ee2fbf9c61c27377511ee8d4c
Successfully built swifter


In [39]:
!pip install my_module

  Created wheel for my-module: filename=my_module-1.6.2-py3-none-any.whl size=1427 sha256=12d287fa025c1f43bf92ea54643e9a0965ad8a9d34007f43aeaa29a227ca279e
  Stored in directory: c:\users\elitebook\appdata\local\pip\cache\wheels\f7\ed\98\51de11048a1c870d648ccd0344bd11c620a8c020ba3b0016f3
Successfully built my-module


In [41]:
!pip install Vector

In [11]:
!pip install Vector

In [14]:
!pip install sentence_transformers